# Program memory: malloc; free; 2D arrays
_COSC 208, Introduction to Computer Systems, 2023-02-24_

## Announcements
* Programming project 2 due Thursday @ 11pm

## Outline
* Warm-up
* malloc
* free
* 2D arrays

## Warm-up
* Q1: _What does the following program output?_

In [21]:
#include <stdio.h>
int main() {
    int numsA[] = {1, 2, 4, 8, 16, 32, 64};
    int *numsB = numsA + 3;
    printf("%d %d\n", *numsA, *numsB);
    numsB += 1;
    *numsB += 1;
    printf("%d %d %d %d\n", numsA[2], numsA[3], numsA[4], numsA[5]);
    *(numsB - 1) = *(numsA + 1);
    numsB[1] += 1;
    printf("%d %d %d %d\n", numsA[2], numsA[3], numsA[4], numsA[5]);
}

1 8
4 8 17 32
4 2 17 33


<p style="height:5em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time take a few deep breaths to reduce stress.

## malloc

* `void* malloc(unsigned int size)`
* Memory allocated on the heap persists until explicitly freed
* When to malloc?
    * When the amount of space required is not known until runtime
    * When a value must remain in memory even after returning from a function
* How much to malloc?
    * Use `sizeof` and a type: e.g., `sizeof(int)`
    * How much to malloc for an array? — multiply sizeof(type) by number of elements in array

* Q2: _Write a function called `duplicate` that takes a string (i.e., an array of `char`) as a parameter and returns a copy of that string stored on the heap._

In [11]:
#include <stdlib.h>
#include <string.h>
char *duplicate(char orig[]) {
    char *copy = malloc(sizeof(char) * (strlen(orig) + 1));
    for (int i = 0; i <= strlen(orig); i++) {
        copy[i] = orig[i];
    }
    // Could replace for loop with: strcpy(copy, orig);
    return copy;
}
// Testing
#include <assert.h>
int main() {
    char *copy = duplicate("abc 123");
    assert(strcmp(copy, "abc 123") == 0);
}

<div style="page-break-after:always;"></div>

* Q3: _Write a function called `range` that behaves similar to the `range` function in Python. Your function should take an unsigned integer (`length`) as a parameter, and return a dynamically allocated array with `length` unsigned integers. The array should be populated with the values 0 through `length-1`._

In [10]:
#include <stdlib.h>
unsigned int *range(unsigned int length) {
    unsigned int *nums = malloc(sizeof(unsigned int) * length);
    for (int i = 0; i < length; i++) {
        nums[i] = i;
    }
    return nums;
}
// Testing
#include <assert.h>
int main() {
    unsigned int *result = range(3);
    assert(result[0] == 0 && result[1] == 1 && result[2] == 2);
}

<p style="height:25em;"></p>

🛑 **STOP here** after completing the above questions; if you have extra time take a few deep breaths to reduce stress.

## free

* `void free(void *block)`
* When to free? — when a value stored on the heap is no longer needed
    * Free memory regions as soon as you are done
    * Do not read/write the memory location after it has been freed!

* _What memory deallocation mistake has been made in each of the following programs?_

In [16]:
// Q4
#include <stdlib.h>
int main() {
    int *ptrA = malloc(sizeof(int) * 3);
    int *ptrB = ptrA;
    free(ptrA);
    free(ptrB);
}

[C kernel] Executable exited with code -6

_Double free_

In [15]:
// Q5
#include <stdlib.h>
int main() {
    int *ptr = malloc(sizeof(int) * 3);
    ptr[0] = 1;
    free(ptr);
    ptr[1] = 2;
}

_Use after free_

In [14]:
// Q6
#include <stdlib.h>
int main() {
    int *ptr = malloc(sizeof(int) * 3);
    ptr++;
    free(ptr);
}

[C kernel] Executable exited with code -6

_Not freeing from beginning of allocated region_

In [13]:
// Q7
#include <stdlib.h>
int main() {
    int *ptrA = malloc(sizeof(int) * 3);
    int *ptrB = ptrA;
    ptrA[0] = 0;
    ptrB[1] = 1;
    free(ptrA);
    ptrB[2] = 2;
}

_Use after free_

🛑 **STOP here** after completing the above questions; if you have extra time take a few deep breaths to reduce stress.

## 2D arrays on the stack
* Assume we wanted to create an array to represent a calendar:
    ```
    +----+----+----+----+----+----+----+
    |  1 |  2 |  3 |  4 |  5 |  6 |  7 |
    +----+----+----+----+----+----+----+
    |  8 |  9 | 10 | 11 | 12 | 13 | 14 |
    +----+----+----+----+----+----+----+
    | 15 | 16 | 17 | 18 | 19 | 20 | 21 |
    +----+----+----+----+----+----+----+
    | 22 | 23 | 24 | 25 | 26 | 27 | 28 |
    +----+----+----+----+----+----+----+
    ```
    * 4 rows = 4 weeks; 7 columns = 7 days in a week
* Q8: _Write a function called `fill_calendar` that creates a 2D array on the stack and populates the array with the appropriate values._


In [19]:
void fill_calendar() {
    int calendar[4][7];
    int day = 1;
    for (int week = 0; week < 4; week++) {
        for (int dow = 0; dow < 7; dow++) {
            calendar[week][dow] = day;
            day++;
        }
    }
}
int main() {
    fill_calendar();
}

<p style="height:30em;"></p>

🛑 **STOP here** after completing the above question; if you have extra time take a few deep breaths to reduce stress.

<div style="page-break-after:always;"></div>

* _Can we use a formula to compute the day based on week and day-of-week (dow)?_
    ```C
    day = week * 7 + dow + 1;
    ```
* _Can we return the `calendar` array from this function?_ – no, it is stored in the stack and will go away when the function returns

## 2D arrays as linear arrays

* Can also think of a calendar as fully linear, like a number line or time line

* Q9: _Fill-in the blanks to complete the `fill_calendar_linear` function which creates a fully linear calendar._
    ```C
    void fill_calendar_linear() {
        int calendar[_____]; // 4 * 7
        int day = 1;
        for (int week = 0; week < 4; week++) {
            for (int dow = 0; dow < 7; dow++) {
                calendar[__________________] = day; // week * 7 + dow
                day++;
            }
        }
    }
    ```

* 2D array indices "map to" 1D array indices
    ```
     (0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (1,0) (1,1) (1,2) (1,3)  ...  (3,6) (3,7)
    +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
    |  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |  9  |  10 |  11 |     |  27 |  28 |
    +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+
    ```
* _Can we the change code such that the function can safely return `calendar`?_

In [ ]:
int *fill_calendar_linear_heap() {
    int *calendar = malloc(sizeof(int) * 28);
    ...
    return calendar;
}

* Only need a single call to malloc 😀
* Determining indices is hard 😕